## Image Preparation For Convolutional Neural Networks

    We'll be using Kaggle's Dogs vs. Cats dataset

In [12]:
# download dataset from: https://www.kaggle.com/c/dogs-vs-cats/data
import kaggle
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.competition_download_file('dogs-vs-cats', 'train.zip')
# then I moved the file to the folder 'dogs-cats-data/'

In [28]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import glob
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

2022-04-04 21:36:08.014852: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ohpc/pub/mpi/openmpi3-gnu7/3.1.0/lib:/opt/ohpc/pub/compiler/gcc/7.3.0/lib64:/usr/local/ncarg/lib:/opt/intel/compilers_and_libraries_2020.1.217/linux/compiler/lib/intel64_lin:/opt/intel/compilers_and_libraries_2020.1.217/linux/mpi/intel64/libfabric/lib:/opt/intel/compilers_and_libraries_2020.1.217/linux/mpi/intel64/lib/release:/opt/intel/compilers_and_libraries_2020.1.217/linux/mpi/intel64/lib:/opt/intel/compilers_and_libraries_2020.1.217/linux/ipp/lib/intel64:/opt/intel/compilers_and_libraries_2020.1.217/linux/mkl/lib/intel64_lin:/opt/intel/compilers_and_libraries_2020.1.217/linux/tbb/lib/intel64/gcc4.8:/opt/intel/compilers_and_libraries_2020.1.217/linux/daal/lib/intel64_lin:/opt/intel/debugger_2020/python/intel64/lib:/opt/intel/debugger

In [45]:
# Organize data into train, valid, test dirs
os.chdir('/home/samanthalof/git/Learning/keras_tf_deeplizard/dogs-cats-data')
if os.path.isdir('train/dog') is False:
    os.makedirs('train/dog')
    os.makedirs('train/cat')
    os.makedirs('valid/dog')
    os.makedirs('valid/cat')
    os.makedirs('test/dog')
    os.makedirs('test/cat')
    
for i in random.sample(glob.glob('outros/cat*'), 500):
    shutil.move(i, 'train/cat')      
for i in random.sample(glob.glob('outros/dog*'), 500):
    shutil.move(i, 'train/dog')
for i in random.sample(glob.glob('outros/cat*'), 100):
    shutil.move(i, 'valid/cat')        
for i in random.sample(glob.glob('outros/dog*'), 100):
    shutil.move(i, 'valid/dog')
for i in random.sample(glob.glob('outros/cat*'), 50):
    shutil.move(i, 'test/cat')      
for i in random.sample(glob.glob('outros/dog*'), 50):
    shutil.move(i, 'test/dog')

os.chdir('../')

In [47]:
train_path = 'dogs-cats-data/train'
valid_path = 'dogs-cats-data/valid'
test_path = 'dogs-cats-data/test'

In [48]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=train_path, target_size=(224,224), classes=['cat', 'dog'], batch_size=10)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=valid_path, target_size=(224,224), classes=['cat', 'dog'], batch_size=10)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=test_path, target_size=(224,224), classes=['cat', 'dog'], batch_size=10, shuffle=False)

Found 1000 images belonging to 2 classes.
Found 200 images belonging to 2 classes.
Found 100 images belonging to 2 classes.


In [51]:
assert train_batches.n == 1000
assert valid_batches.n == 200
assert test_batches.n == 100
assert train_batches.num_classes == valid_batches.num_classes == test_batches.num_classes == 2